<a href="https://colab.research.google.com/github/rileycsv/PDF-to-CSV-Infoceutical-Extractor/blob/main/PDFtoCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ---------------------------------------------
# 1. Install libraries
# ---------------------------------------------
!pip install PyMuPDF pandas --quiet

import fitz
import pandas as pd
import re
from google.colab import files

# ---------------------------------------------
# 2. Upload your PDF
# ---------------------------------------------
print("⬆️ Upload your Infoceuticals PDF…")
uploaded = files.upload()
PDF_NAME = next(iter(uploaded))
doc = fitz.open(PDF_NAME)

# ---------------------------------------------
# 3. Update tag types
# ---------------------------------------------
END_TAGS = ("Imprint", "PEMF", "Rife", "Nosode", "Audible")
TITLE_RX = re.compile(rf"^(.*?)\s+\[({'|'.join(END_TAGS)})\]\s*$", re.IGNORECASE)

def clean(txt: str) -> str:
    return re.sub(r"[^\w\s:/.\-]", "", txt).strip()

entries = []
seen = set()

# ---------------------------------------------
# 4. Parse each page
# ---------------------------------------------
for page in doc:
    # get all clickable hyperlinks on the page
    links = [(fitz.Rect(l["from"]), l["uri"]) for l in page.get_links() if l.get("uri")]

    # extract all visible text lines (ignore figures/images)
    lines = []
    for b in page.get_text("dict")["blocks"]:
        if "lines" not in b:
            continue  # skip non-text blocks
        for l in b["lines"]:
            y = l["bbox"][1]
            text = "".join(s["text"] for s in l["spans"]).strip()
            if text:
                lines.append((y, text))
    lines.sort(key=lambda tup: tup[0])  # sort top-to-bottom

    # walk through each line and match titles
    i = 0
    while i < len(lines):
        y, txt = lines[i]
        m = TITLE_RX.match(txt)
        if m:
            title = clean(txt)
            if title in seen:
                i += 1
                continue

            # get hyperlink associated with this title line
            url = ""
            for rect, uri in links:
                if rect.y0 - 1 <= y <= rect.y1 + 1:
                    url = uri
                    break

            if not url:
                i += 1
                continue  # skip if title is not hyperlinked

            # get all description lines after this title
            desc_lines = []
            j = i + 1
            while j < len(lines) and not TITLE_RX.match(lines[j][1]):
                desc_lines.append(lines[j][1])
                j += 1

            description = clean(" ".join(desc_lines))
            entries.append([title, description, url])
            seen.add(title)
            i = j
        else:
            i += 1

# ---------------------------------------------
# 5. Export as CSV with clickable links
# ---------------------------------------------
def sanitize(text):
    if not text:
        return ""

    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore")

    # Remove unwanted invisible or weird characters
    BAD_CHARS = ['©', '®', '™', '\u200b', '\u202a', '\ufeff']
    for char in BAD_CHARS:
        text = text.replace(char, '')

    # Clean up formatting
    text = text.replace("=--", "-").replace("--", "-")
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('"', "'").replace('\n', ' ').strip()

    if text.startswith(("=", "+", "-", "@")):
        text = "\t" + text

    return text

def hyperlink(url):
    return f'=HYPERLINK("{url}", "{url}")'

# Clean up and add clickable links
df = pd.DataFrame(entries, columns=["Title", "Description", "URL"])
df["Title"] = df["Title"].apply(sanitize)
df["Description"] = df["Description"].apply(sanitize)
df["Link (clickable)"] = df["URL"].apply(hyperlink)

# Drop raw URL (optional for clarity)
df = df[["Title", "Description", "Link (clickable)"]]

# Save CSV
csv_name = "infoceuticals_clickable_clean.csv"
df.to_csv(csv_name, index=False, quoting=1, encoding='utf-8')

print(f"✅ Cleaned and clickable. Saved {len(df)} entries.")
files.download(csv_name)

⬆️ Upload your Infoceuticals PDF…


Saving all_infoceuticals_alphabetic_en.pdf to all_infoceuticals_alphabetic_en (3).pdf
✅ Cleaned and clickable. Saved 3681 entries.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# ---------------------------------------------
# 1. Install libraries
# ---------------------------------------------
!pip install PyMuPDF pandas --quiet

import fitz
import pandas as pd
import re
from google.colab import files
import unicodedata

# ---------------------------------------------
# 2. Upload your PDF
# ---------------------------------------------
print("⬆️ Upload your Infoceuticals PDF…")
uploaded = files.upload()
PDF_NAME = next(iter(uploaded))
doc = fitz.open(PDF_NAME)

# ---------------------------------------------
# 3. Update tag types
# ---------------------------------------------
END_TAGS = ("Imprint", "PEMF", "Rife", "Nosode", "Audible")
TITLE_RX = re.compile(rf"^(.*?)\s+\[({'|'.join(END_TAGS)})\]\s*$", re.IGNORECASE)

def clean(txt: str) -> str:
    return re.sub(r"[^\w\s:/.\-]", "", txt).strip()

entries = []
seen = set()

# ---------------------------------------------
# 4. Parse each page
# ---------------------------------------------
for page in doc:
    # get all clickable hyperlinks on the page
    links = [(fitz.Rect(l["from"]), l["uri"]) for l in page.get_links() if l.get("uri")]

    # extract all visible text lines (ignore figures/images)
    lines = []
    for b in page.get_text("dict")["blocks"]:
        if "lines" not in b:
            continue  # skip non-text blocks
        for l in b["lines"]:
            y = l["bbox"][1]
            text = "".join(s["text"] for s in l["spans"]).strip()
            if text:
                lines.append((y, text))
    lines.sort(key=lambda tup: tup[0])  # sort top-to-bottom

    # walk through each line and match titles
    i = 0
    while i < len(lines):
        y, txt = lines[i]
        m = TITLE_RX.match(txt)
        if m:
            title = clean(txt)
            if title in seen:
                i += 1
                continue

            # get hyperlink associated with this title line
            url = ""
            for rect, uri in links:
                if rect.y0 - 1 <= y <= rect.y1 + 1:
                    url = uri
                    break

            if not url:
                i += 1
                continue  # skip if title is not hyperlinked

            # get all description lines after this title
            desc_lines = []
            j = i + 1
            while j < len(lines) and not TITLE_RX.match(lines[j][1]):
                desc_lines.append(lines[j][1])
                j += 1

            description = clean(" ".join(desc_lines))
            entries.append([title, description, url])
            seen.add(title)
            i = j
        else:
            i += 1

# ---------------------------------------------
# 5. Export as CSV with clickable links
# ---------------------------------------------

def sanitize(text):
    if not text:
        return ""

    # Remove common junk symbols
    BAD_CHARS = ['©', '®', '™', '\u200b', '\u202a', '\ufeff']  # invisible, RTL, BOM, etc.
    for char in BAD_CHARS:
        text = text.replace(char, '')

    # Clean Excel-breaking characters
    text = text.replace("=--", "-").replace("--", "-")
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('"', "'").replace('\n', ' ').strip()

    # Prevent formula execution in Excel
    if text.startswith(("=", "+", "-", "@")):
        text = "\t" + text

    return text

def hyperlink(url):
    return f'=HYPERLINK("{url}", "{url}")'

# Clean up and add clickable links
df = pd.DataFrame(entries, columns=["Title", "Description", "URL"])
df["Title"] = df["Title"].apply(sanitize)
df["Description"] = df["Description"].apply(sanitize)
df["Link (clickable)"] = df["URL"].apply(hyperlink)

# Drop raw URL (optional for clarity)
df = df[["Title", "Description", "Link (clickable)"]]

# Save CSV
csv_name = "infoceuticals_clickable_clean.csv"
df.to_csv(csv_name, index=False, quoting=1, encoding='utf-8')

print(f"✅ Cleaned and clickable. Saved {len(df)} entries.")
files.download(csv_name)

⬆️ Upload your Infoceuticals PDF…


Saving all_infoceuticals_alphabetic_en.pdf to all_infoceuticals_alphabetic_en (2).pdf
✅ Cleaned and clickable. Saved 3681 entries.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>